In [69]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [70]:
#PROGRAM TO REDUCE MEMORY SIZE OF EACH INPUT FILE
print('WRITING PROGRAM TO REDUCE FILE SIZE....')
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

#CONVERT AND RETURN NEW FILE

def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

print('PROGRAM COMPLETE....')


In [71]:
#CONVERT ALL AVAILABLE FILES
print('CONVERTING ALL INPUT FILES....')

print('-' * 80)
print('train')
app_train = import_data('../input/application_train.csv')

print('-' * 80)
print('test')
app_test = import_data('../input/application_test.csv')

print('-' * 80)
print('bureau_balance')
bur_bal = import_data('../input/bureau_balance.csv')

print('-' * 80)
print('bureau')
bur = import_data('../input/bureau.csv')

print('-' * 80)
print('credit_card_balance')
credit = import_data('../input/credit_card_balance.csv')

print('-' * 80)
print('installments_payments')
install = import_data('../input/installments_payments.csv')

print('-' * 80)
print('pos_cash_balance')
pos = import_data('../input/POS_CASH_balance.csv')

print('-' * 80)
print('previous_application')
prev = import_data('../input/previous_application.csv')

print('CONVERSION FOR ALL FILES COMPLETE....')


In [72]:
#SAVING A COPY OF TEST FILE FOR LATER USE
print('SAVING A COPY OF TEST FILE FOR LATER USE....')

app_test1 = app_test

print('SAVING COMPLETE....')

In [73]:
#WRITE A PROGRAM TO LABEL CODE THE CATEGORY COLUMNS
print('WRITING A PROGRAM TO LABEL CODE CATEGORY COL....')

def encode(df, nan_as_category = True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return new_columns,df

print('PROGRAM COMPLETE....')

In [74]:
#IMPUTE MISSING VALUES IN OCCUPATION BASED ON SALARY SLAB

print('IMPUTING OCCUPATION TYPE BASED ON SALARY SLAB IN TRAIN & TEST DATA....')

#FINDING MISSING DATA LOCATIONS
miss = app_train['OCCUPATION_TYPE'].isnull()
miss1 = app_test['OCCUPATION_TYPE'].isnull()

#CREATE AN EMPTY LIST TO CAPTURE NEW OCCUPATION DATA
test = []
test1 = []

#FIND THE INCOME FROM MISSING DATA SET
df = app_train.loc[miss,'AMT_INCOME_TOTAL']
df_test = app_test.loc[miss1, 'AMT_INCOME_TOTAL']

#IMPUTE BASED ON MEAN INCOME FOR EACH TYPE OF OCCUPATION
for x in df:
    if  x < 130791:
        test.append('Cleaning staff')
    elif (x>130791) & (x< 133228):
        test.append('Low-skill Laborers')
    elif (x>133228) & (x< 138397):
        test.append('Cooking staff')
    elif (x>138397) & (x <144273):
        test.append('Waiters/barmen staff')
    elif (x>144273) & (x < 149663):
        test.append('Security staff')
    elif (x>149663) & (x < 149710):
        test.append('Medicine staff')
    elif (x>149710) & (x <152303):
        test.append('Sales staff')
    elif (x>152303) & (x <160542):
        test.append('Secretaries')
    elif (x>160542) & (x <166357):
        test.append('Laborers')
    elif (x>166357) & (x <172657):
        test.append('Core staff')
    elif (x>172657) & (x <182335):
        test.append('Private service staff')
    elif (x>182335) & (x <182842):
        test.append('High skill tech staff')
    elif (x>182842) & (x <187012):
        test.append('Drivers')
    elif (x>187012) & (x <188916):
        test.append('HR staff')
    elif (x>188916) & (x <194578):
        test.append('Accountants')
    elif (x>194578) & (x <195004):
        test.append('Realty agents')
    elif (x>195004) & (x <213446):
        test.append('IT staff')
    elif (x>213446):
        test.append('Managers')


In [75]:
#DO THE SAME FOR TEST DATA
for x in df_test:
    if  x < 130791:
        test1.append('Cleaning staff')
    elif (x>130791) & (x< 133228):
        test1.append('Low-skill Laborers')
    elif (x>133228) & (x< 138397):
        test1.append('Cooking staff')
    elif (x>138397) & (x <144273):
        test1.append('Waiters/barmen staff')
    elif (x>144273) & (x < 149663):
        test1.append('Security staff')
    elif (x>149663) & (x < 149710):
        test1.append('Medicine staff')
    elif (x>149710) & (x <152303):
        test1.append('Sales staff')
    elif (x>152303) & (x <160542):
        test1.append('Secretaries')
    elif (x>160542) & (x <166357):
        test1.append('Laborers')
    elif (x>166357) & (x <172657):
        test1.append('Core staff')
    elif (x>172657) & (x <182335):
        test1.append('Private service staff')
    elif (x>182335) & (x <182842):
        test1.append('High skill tech staff')
    elif (x>182842) & (x <187012):
        test1.append('Drivers')
    elif (x>187012) & (x <188916):
        test1.append('HR staff')
    elif (x>188916) & (x <194578):
        test1.append('Accountants')
    elif (x>194578) & (x <195004):
        test1.append('Realty agents')
    elif (x>195004) & (x <213446):
        test1.append('IT staff')
    elif (x>213446):
        test1.append('Managers')

#ADD THE CAPTURED OCCUPATION DATA  IN MISSING LOCATIONS
app_train.loc[miss,'OCCUPATION_TYPE'] = test
app_test.loc[miss1,'OCCUPATION_TYPE'] = test1

print('IMPUTING IS COMPLETE FOR OCCUPATION TYPE....')


In [76]:
#BINARIZING Y/N COLUMNS FOR TRAIN & TEST DATA
print('BINARIZING Y/N COLUMNS FOR TRAIN AND TEST DATA....')

for df in (app_train,app_test):
    for bin_feature in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
            df[bin_feature], uniques = pd.factorize(df[bin_feature])

print('BINARUZING COMPLETE....')

#FEATURE ENGINEERING
print('FEATURE ENGINEERING FOR TRAIN & TEST....')

for df in (app_train,app_test):
    df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace= True)
    df['DAYS_EMPLOYED_PERC'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
    df['DAYS_REG_PERC'] = df['DAYS_REGISTRATION'] / df['DAYS_BIRTH']
    df['DAYS_ID_PERC'] = df['DAYS_ID_PUBLISH'] / df['DAYS_REGISTRATION']
    df['INCOME_CREDIT_PERC'] = df['AMT_INCOME_TOTAL'] / df['AMT_CREDIT']
    df['INCOME_PER_PERSON'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']
    df['ANNUITY_INCOME_PERC'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
    df['LOAN_INCOME_RATIO'] = df['AMT_CREDIT'] / df['AMT_INCOME_TOTAL']
    df['CONSUMER_GOODS_RATIO'] = df['AMT_CREDIT'] / df['AMT_GOODS_PRICE']
    df['ANNUITY LENGTH'] = df['AMT_CREDIT'] / df['AMT_ANNUITY']
    df['ANN_LENGTH_EMPLOYED_RATIO'] = df['ANNUITY LENGTH'] / df['DAYS_EMPLOYED']
    df['WORKING_LIFE_RATIO'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
    df['TOTAL_DOCS_SUBMITTED'] = df.loc[:, df.columns.str.contains('FLAG_DOCUMENT')].sum(axis=1)
    
print('FEATURE ENGINEERING COMPLETE....')


In [77]:
#REPLACING ODD VALUES WITH NAN
print('REPLACE ODD VALUES WITH NAN....')

prev['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace= True)
prev['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace= True)
prev['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace= True)
prev['DAYS_LAST_DUE'].replace(365243, np.nan, inplace= True)
prev['DAYS_TERMINATION'].replace(365243, np.nan, inplace= True)

#FEATURE ENGINEERING
print('FEATURE ENGINEERING FOR PREVIOUS APP....')

prev['ASK_AMT_RATIO'] = prev['AMT_APPLICATION'] / prev['AMT_CREDIT']
prev['APPROVED'] = (prev['NAME_CONTRACT_STATUS'] == 'Approved').astype(int)
prev['REFUSED'] = (prev['NAME_CONTRACT_STATUS'] == 'Refused').astype(int)
prev['APP_CREDIT_PERC'] = prev['AMT_APPLICATION'] / prev['AMT_CREDIT']

#feature engineering for numerical columns
agg = {
    'AMT_ANNUITY': ['mean','max','sum','var','median'],
    'AMT_APPLICATION': ['mean','max','sum','var','median'],
    'AMT_CREDIT': ['mean','max','sum','var','median'],
    'APP_CREDIT_PERC': ['mean','max','sum','var','median'],
    'AMT_DOWN_PAYMENT': ['mean','max','sum','var','median'],
    'AMT_GOODS_PRICE': ['mean','max','sum','var','median'],
    'HOUR_APPR_PROCESS_START': ['mean','max','sum','var','median'],
    'RATE_DOWN_PAYMENT': ['mean','max','sum','var','median'],
    'DAYS_DECISION': ['mean','max','sum','var','median'],
    'CNT_PAYMENT': ['mean', 'sum','max','var','median'],
}
    
prev=prev.groupby('SK_ID_CURR').agg(agg)
prev.reset_index(inplace=True)
prev.columns = pd.Index([e[0] + "_" + e[1].upper() for e in prev.columns.tolist()])
prev.rename(columns={'SK_ID_CURR_':'SK_ID_CURR'}, inplace=True)

print('FEATURE ENGINEERING FOR PREV COMPLETE')

In [78]:
prev.head()

In [79]:
print('FEATURE ENGINEERING FOR CREDIT.....')
credit['AMT_DRAW'] = credit['AMT_DRAWINGS_ATM_CURRENT'] + credit['AMT_DRAWINGS_CURRENT'] +credit['AMT_DRAWINGS_OTHER_CURRENT'] + credit['AMT_DRAWINGS_POS_CURRENT']
credit['CNT_DRAW'] = credit['CNT_DRAWINGS_ATM_CURRENT'] + credit['CNT_DRAWINGS_CURRENT'] + credit['CNT_DRAWINGS_OTHER_CURRENT'] + credit['CNT_DRAWINGS_POS_CURRENT']
credit['cr_util'] = (credit['AMT_BALANCE']*100) / credit['AMT_CREDIT_LIMIT_ACTUAL']
credit['PAID%'] = (credit['AMT_PAYMENT_TOTAL_CURRENT']*100) / credit['AMT_TOTAL_RECEIVABLE']

#REPLACING ALL INF AND -INF WITH NAN
credit=credit.replace(np.inf, np.nan)
credit=credit.replace(-np.inf, np.nan)

#GROUPING AND AGGREGATING
credit=credit.groupby('SK_ID_CURR').agg(['mean','sum','max','min','var','median'])
credit.reset_index(inplace=True)
credit.columns = pd.Index([e[0] + "_" + e[1].upper() for e in credit.columns.tolist()])
del credit['SK_ID_PREV_MEAN']
del credit['SK_ID_PREV_MAX']
del credit['SK_ID_PREV_MIN']
del credit['SK_ID_PREV_SUM']
del credit['SK_ID_PREV_VAR']
del credit['SK_ID_PREV_MEDIAN']
           

print('FEATURE ENGINEERING FOR CREDIT COMPLETE')

In [80]:
credit = credit.rename(columns={'SK_ID_CURR_':'SK_ID_CURR'})

In [81]:
credit.head()

In [82]:
# FEATURE ENGINEERING FOR INSTALLMENT
print('FEATURE ENGINEERING FOR INSTALLMENT.....')

install['PAYMENT_PERC'] = install['AMT_PAYMENT'] / install['AMT_INSTALMENT']
install['PAYMENT_DIFF'] = install['AMT_INSTALMENT'] - install['AMT_PAYMENT']
install['DAYS_INSTALLMENT_LATE'] = install['DAYS_INSTALMENT'] - install['DAYS_ENTRY_PAYMENT']
install['PAYMENT_DISCREPANCY'] = install['AMT_INSTALMENT'] - install['AMT_PAYMENT']
install['DPD'] = install['DAYS_ENTRY_PAYMENT'] - install['DAYS_INSTALMENT']
install['DBD'] = install['DAYS_INSTALMENT'] - install['DAYS_ENTRY_PAYMENT']
install['DPD'] = install['DPD'].apply(lambda x: x if x > 0 else 0)
install['DBD'] = install['DBD'].apply(lambda x: x if x > 0 else 0)

#GROUPING AND AGGREGATING
install = install.groupby('SK_ID_CURR').agg(['mean','sum','max','min','var','median'])
install.columns = pd.Index([e[0] + "_" + e[1].upper() for e in install.columns.tolist()])
install.reset_index(inplace=True)
del install['SK_ID_PREV_MEAN']
del install['SK_ID_PREV_SUM']
del install['SK_ID_PREV_MAX']
del install['SK_ID_PREV_MIN']
del install['SK_ID_PREV_VAR']
del install['SK_ID_PREV_MEDIAN']
print('FEATURE ENGINEERING FOR INSTALL COMPLETED....')

In [83]:
install.head()

In [84]:
# FEATURE ENGINEERING FOR POS
print('FEATURE ENGINEERING FOR POS.....')
pos=pos.groupby('SK_ID_CURR').agg(['mean','sum','max','var','min','median'])
pos.columns = pd.Index([e[0] + "_" + e[1].upper() for e in pos.columns.tolist()])
pos.reset_index(inplace=True)
del pos['SK_ID_PREV_MEAN']
del pos['SK_ID_PREV_MAX']
del pos['SK_ID_PREV_VAR']
del pos['SK_ID_PREV_SUM']
del pos['SK_ID_PREV_MIN']
del pos['SK_ID_PREV_MEDIAN']

print('FEATURE ENGINEERING FOR POS COMPLETED....')

In [85]:
pos.head()

In [86]:
print('FEATURE ENGINEERING FOR BUR AND BAL.....')
bal_agg = {'MONTHS_BALANCE': ['min', 'max','size','mean','var','median']}
bur_bal = bur_bal.groupby('SK_ID_BUREAU').agg(bal_agg)
bur_bal.columns = pd.Index([e[0] + "_" + e[1].upper() for e in bur_bal.columns.tolist()])
bur_bal = bur.join(bur_bal, how='left', on='SK_ID_BUREAU')
bur_bal.drop(columns= 'SK_ID_BUREAU', inplace= True)

a = {
        'DAYS_CREDIT': ['mean','max','sum','min','var','median'],
        'CREDIT_DAY_OVERDUE': ['max','min','mean','sum','var','median'],
        'DAYS_CREDIT_ENDDATE': ['mean','max','min','sum','var','median'],
        'AMT_CREDIT_MAX_OVERDUE': ['mean','max','min','sum','var','median'],
        'CNT_CREDIT_PROLONG': ['sum','max','min','mean','var','median'],
        'AMT_CREDIT_SUM': ['mean', 'sum','max','min','var','median'],
        'AMT_CREDIT_SUM_DEBT': ['mean', 'sum','max','min','var','median'],
        'AMT_CREDIT_SUM_OVERDUE': ['mean','max','min','var','sum','median'],
        'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum','max','min','var','median'],
        'DAYS_CREDIT_UPDATE': ['mean','max','min','var','sum','median'],
        'AMT_ANNUITY': ['mean','max','min','sum','var','median'],
        'MONTHS_BALANCE_MIN': ['min'],
        'MONTHS_BALANCE_MAX': ['max'],
        'MONTHS_BALANCE_SIZE': ['mean']
    }
bur_bal = bur_bal.groupby('SK_ID_CURR').agg(a)
bur_bal.columns = pd.Index(['BUR_' + e[0] + "_" + e[1].upper() for e in bur_bal.columns.tolist()])
bur_bal.reset_index(inplace=True)
bur_bal.head()
print('FEATURE ENGINEERING FOR BUR AND BAL COMPLETED.....')


In [87]:
bur_bal.head()

In [88]:
#joinging all datas on sk_id_curr
print('JOIN ALL DATAS ON SK_ID_CURR....')

app_train = app_train.merge(right=bur_bal, how='left', on='SK_ID_CURR')
app_test = app_test.merge(right=bur_bal, how='left', on='SK_ID_CURR')

app_train = app_train.merge(right=prev, how='left', on='SK_ID_CURR')
app_test = app_test.merge(right=prev, how='left', on='SK_ID_CURR')

app_train = app_train.merge(right=pos, how='left', on='SK_ID_CURR')
app_test = app_test.merge(right=pos, how='left', on='SK_ID_CURR')

app_train = app_train.merge(right=credit, how='left', on='SK_ID_CURR')
app_test = app_test.merge(right=credit, how='left', on='SK_ID_CURR')

app_train = app_train.merge(right=install, how='left', on='SK_ID_CURR')
app_test = app_test.merge(right=install, how='left', on='SK_ID_CURR')

del app_train['SK_ID_CURR']
del app_test['SK_ID_CURR']

print('JOIN ALL DATAS ON SK_ID_CURR COMPLETE....')


In [89]:
print(app_train.shape)
print(app_test.shape)

In [90]:
#PREPARING THE DATA FOR PREPROCESSING
train = app_train.drop(['TARGET'],axis=1)
test = app_test
target = app_train['TARGET']
del app_train['TARGET']


In [91]:
#PROGRAM TO LABEL ENCODE CATEGORICAL COL
print('LABEL ENCODING ALL CATEGORY COL......')

from sklearn.preprocessing import LabelEncoder
var_mod = train.select_dtypes(include = 'object').columns.tolist()
num = train.columns.tolist()
le = LabelEncoder()
for i in var_mod:
    train[i] = le.fit_transform(train[i].astype('str'))
    test[i] = le.fit_transform(test[i].astype('str'))
    
print('LABEL ENCODING ALL CATEGORY COL COMPLETE......')


In [92]:
print(test.shape)
print(train.shape)

In [93]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import gc

folds = KFold(n_splits=5, shuffle=True, random_state=546789)
oof_preds = np.zeros(train.shape[0])
sub_preds = np.zeros(test.shape[0])
feature_importance_df = pd.DataFrame()


for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train, target)):
    train_x, train_y = train.iloc[train_idx], target.iloc[train_idx]
    valid_x, valid_y = train.iloc[valid_idx], target.iloc[valid_idx]

    #SETTING THE CLASSIFIER
    print('PREPARING THE LGBM CLASSIFIER.....')

    clf = LGBMClassifier(
            n_estimators=10000,
            learning_rate=0.03,
            num_leaves=45,
            colsample_bytree=0.9,
            subsample=0.8,
            max_depth=8,
            reg_alpha=.8,
            reg_lambda=.7,
            min_split_gain=.01,
            min_child_weight=300,
            silent=-1,
            verbose=-1,
            class_weight='balanced',
            boosting_type = 'dart',
            bagging_fraction = 0.6,
            bagging_freq = 100
            )
    print('CLASSIFIER IS TUNNED....')
    
    clf.fit(train_x, train_y, 
            eval_set= [(train_x, train_y), (valid_x, valid_y)], 
            eval_metric='auc', verbose=100, early_stopping_rounds=100, feature_name = num, categorical_feature=var_mod #30
           )
    
    oof_preds[valid_idx] = clf.predict_proba(valid_x, num_iteration=clf.best_iteration_)[:, 1]
    sub_preds += clf.predict_proba(test, num_iteration=clf.best_iteration_)[:, 1] / folds.n_splits
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = train_x.columns.tolist()
    fold_importance_df["importance"] = clf.feature_importances_
    fold_importance_df["fold"] = n_fold + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(valid_y, oof_preds[valid_idx])))
    del train_x, train_y, valid_x, valid_y
    gc.collect()

In [94]:
# #print auc score
# print('AUC : %.6f' % (roc_auc_score(target, sub_preds)))

#feature selection
best_features = pd.DataFrame()
feature_importance_df.sort_values(by='importance', ascending=False)
cols = fold_importance_df.sort_values(by="importance", ascending=False)[:100].index
best_features = fold_importance_df.loc[(cols)]
best_features.sort_values(by='importance',ascending=False)

In [95]:
#plot top 150
best_features.set_index('feature').plot(kind='barh', figsize=(26,18) )

In [96]:
print('FIT MODEL WITH TOP 100 FEATURES.....')
#cnvert cols to list format
cols = best_features.feature.tolist()

#selecting top 50 cols
train = train[cols]
test = test[cols]

clf.fit(train,target)

In [97]:
print('PREDICTING TEST DATA......')
#predict 
pred = clf.predict_proba(test,num_iteration=clf.best_iteration_)[:,1]

In [98]:
p5 = pd.Series(data=pred)

In [99]:
sub= app_test1
sub['TARGET'] = p5
t2 = sub[['SK_ID_CURR','TARGET']]
t2.head()

In [100]:
print(t2.isnull().sum())
t2.head()

In [50]:
t2.to_csv('submission.csv',index=False)